# Mobile App for Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49](https://en.wikipedia.org/wiki/Lotto_6/49) lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. The [data set](https://github.com/dataquestio/solutions/blob/master/Mission382Solutions.ipynb) has data for 3,665 drawings, dating from 1982 to 2018.

In [13]:
import pandas as pd

## Single Ticket Probability

First, we start off with writing functions to calculate the factorial and the combination.

In [14]:
def factorial(n):
    product = 1
    while n > 1:
        product *= n
        n -= 1
    return product

def combination(n, k):
    return factorial(n) / (factorial(k) * factorial(n - k))

Next, we want to calculate the probability of winning the big prize in the 6/49 lottery. Siz numbers are chosen without replacement, each with a range of 1 - 49.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.


In [27]:
def one_ticket_probability(numbers):
    outcomes = combination(49, 6)
    probability = 1 / outcomes
    probability_percentage = probability * 100
    message = print('The probability of winning is {:f}%'
                    .format(probability_percentage)
                   )
    return message

In [28]:
test = [49, 48, 47, 46, 45, 44]

one_ticket_probability(test)

The probability of winning is 0.000007%


In [29]:
df = pd.read_csv('649.csv')
print(df.shape)
print(df.head(3))
print(df.tail(3))

(3665, 11)
   PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
0      649            1                0  6/12/1982               3   
1      649            2                0  6/19/1982               8   
2      649            3                0  6/26/1982               1   

   NUMBER DRAWN 2  NUMBER DRAWN 3  NUMBER DRAWN 4  NUMBER DRAWN 5  \
0              11              12              14              41   
1              33              36              37              39   
2               6              23              24              27   

   NUMBER DRAWN 6  BONUS NUMBER  
0              43            13  
1              41             9  
2              39            34  
      PRODUCT  DRAW NUMBER  SEQUENCE NUMBER  DRAW DATE  NUMBER DRAWN 1  \
3662      649         3589                0  6/13/2018               6   
3663      649         3590                0  6/16/2018               2   
3664      649         3591                0  6/20/2018              1

In [30]:
# Extract lottery numbers from dataset and create new pandas series with all historic winning
# numbers.
def extract_numbers(row):
    numbers = []
    for column in range(4, 10):
        numbers.append(row[column])
    return numbers

historic_winning_nums = pd.Series(df.apply(extract_numbers, axis=1))

In [31]:
def check_historical_occurence(numbers, historic_winning_nums):
    number_set = set(numbers)
    matches = 0
    for row in historic_winning_nums:
        if number_set == row:
            matches += 1
    print('Number of times your numbers have won in all previous lotterys: {0}'
          .format(matches)
         )
    one_ticket_probability(numbers)
    return

In [32]:
check_historical_occurence([1,2,3,4,5,6], historic_winning_nums)

Number of times your numbers have won in all previous lotterys: 0
The probability of winning is 0.000007%


## Multi-ticket Probability

Many lottery addicts play multiple tickets so we would like to give them feedback on the odds of winning for different scenarios.

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [33]:
def multi_ticket_probability(tickets):
    max_tickets = 13983816 # combination 49 pick 6
    probability = tickets / max_tickets * 100
    message = print('The probability of winning is {:f}%'
                    .format(probability)
                   )
    return message

In [37]:
multi_ticket_probability(100000)

The probability of winning is 0.715112%


In [41]:
def probability_less_6(winning_numbers):
    winning_outcomes = combination(6, winning_numbers)
    k = 6 - winning_numbers
    n = 49 - winning_numbers - 1 # need to subtract 1 to remove winning number
    total_outcomes = winning_outcomes * (factorial(n) / factorial(n - k))
    probability = total_outcomes / 13983816 * 100 # combination pick 6
    message = print('The probability of winning {} is {:f}%'
                .format(probability)
               )
    return message

probability_less_6(5)

The probability of winning is 0.001845%
